In [68]:
# import the necessary packages
import face_recognition
import argparse
import pickle
import cv2
import os
# construct the argument parser and parse the arguments
ap=argparse.ArgumentParser()
ap.add_argument("-e", "--encodings", required=True, help="path to serialized db of facial encodings")
ap.add_argument("-i", "--image", required=True, help="path to input image")
ap.add_argument("-d", "--detection-method", type=str, default="cnn", help="face detection model to use: either `hog` or `cnn`")
args= vars(ap.parse_args)

In [69]:
print(os.getcwd())

C:\Users\sphaniraj


In [70]:
# load the known faces and embeddings
print("loading encodings...")
data = pickle.loads(open("encodings", "rb").read())
# load the input image and convert it from BGR to RGB
image = cv2.imread("examples\example_7.jpg")
#print(image)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# detect the (x, y)-coordinates of the bounding boxes corresponding
# to each face in the input image, then compute the facial embeddings
# for each face
print("recognizing faces...")
boxes = face_recognition.face_locations(rgb,model="detection_method")
encodings = face_recognition.face_encodings(rgb, boxes)
# initialize the list of names for each face detected
names = []

loading encodings...
recognizing faces...


In [71]:
print(boxes)
print(encodings)

[(130, 579, 182, 527), (135, 411, 198, 349), (177, 349, 239, 287)]
[array([-0.14032573,  0.04912601,  0.09384458, -0.0361614 , -0.08144219,
       -0.03211403, -0.10941029, -0.07276662,  0.18814099, -0.05690083,
        0.18693349,  0.02701893, -0.12492654, -0.12306495, -0.05401034,
        0.06274499, -0.151021  , -0.12970726, -0.04641719, -0.08927697,
        0.0064336 , -0.00900209,  0.00228284,  0.00322308, -0.27449802,
       -0.35074183, -0.10380649, -0.13077998,  0.05540316, -0.10423569,
        0.06541584,  0.01790967, -0.23515697, -0.04849654,  0.01549627,
        0.13715407, -0.03178289,  0.04306391,  0.19814789,  0.0176608 ,
       -0.15959024,  0.00060312, -0.00778943,  0.2394703 ,  0.22046188,
        0.04892879,  0.045003  , -0.09955294,  0.16193454, -0.17707454,
        0.11617086,  0.19625317,  0.09942766,  0.00965582,  0.20049885,
       -0.11978413, -0.03334621,  0.18280853, -0.16370313,  0.0781778 ,
        0.01169051,  0.01472355, -0.0493634 , -0.05597164,  0.112809

In [72]:
# loop over the facial embeddings
for encoding in encodings:
    # attempt to match each face in the input image to our known
    # encodings
    matches = face_recognition.compare_faces(data["encodings"],encoding)
    name = "Unknown"
    if True in matches:
        # find the indexes of all matched faces then initialize a
        # dictionary to count the total number of times each face
        # was matched
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            name = data["names"][i]
            counts[name] = counts.get(name, 0) + 1
        # determine the recognized face with the largest number of
        # votes (note: in the event of an unlikely tie Python will
        # select first entry in the dictionary)
        print(counts)
        name = max(counts, key=counts.get)
    # update the list of names
    names.append(name)

{'ian_malcolm': 60}
{'alan_grant': 6, 'ellie_sattler': 2}


In [73]:
print(names)

['ian_malcolm', 'alan_grant', 'Unknown']


In [74]:
print(boxes)

[(130, 579, 182, 527), (135, 411, 198, 349), (177, 349, 239, 287)]


In [75]:
# loop over the recognized faces
for ((top, right, bottom, left), name) in zip(boxes, names):
    # draw the predicted face name on the image
    cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
    y = top - 15 if top - 15 > 15 else top + 15
    cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)
# show the output image
cv2.imshow("Image", image)
cv2.imwrite(f"Sample_7.jpg", image)
k=cv2.waitKey(0)

if k == ord('s'): # wait for 's' key to save and exit
    a=random.randint(0,99)  
    cv2.destroyAllWindows()   